<a href="https://colab.research.google.com/github/richmondvan/melanoma-detection/blob/master/process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

**This cell is mandatory!**

- Imports modules

- Mounts Google Drive

- Sets up constants

In [1]:
# Must be run every time!
import numpy as np
import tensorflow as tf

from google.colab import drive 
drive.mount('/content/gdrive') 

DATASET_FILEPATH = "/content/gdrive/My Drive/Dataset/dataset.zip"
DIRECTORY_PATH = "/content/gdrive/My Drive/Dataset/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


One-time:
Download my data from ISIC

In [0]:
import urllib

urllib.request.urlretrieve("https://isic-archive.com/api/v1/image/download?include=all&filter={%22operator%22:%22and%22,%22operands%22:[{%22operator%22:%22in%22,%22operands%22:[{%22identifier%22:%22meta.datasetId%22,%22type%22:%22objectid%22},[%225a2ecc5e1165975c945942a4%22,%225a2ecc5e1165975c945942a2%22,%225a2ecc5d1165975c94594292%22,%225a2ecc5d1165975c9459428e%22,%225a2ecc5d1165975c94594284%22,%225aaf6f2a116597691367292e%22,%225a2ecc5d1165975c9459427e%22,%225a2ecc5d1165975c9459428a%22]]},{%22operator%22:%22and%22,%22operands%22:[{%22operator%22:%22in%22,%22operands%22:[{%22identifier%22:%22meta.clinical.benign_malignant%22,%22type%22:%22string%22},[%22benign%22,%22malignant%22]]},{%22operator%22:%22in%22,%22operands%22:[{%22identifier%22:%22meta.tags%22,%22type%22:%22string%22},[%22Challenge%202019:%20Training%22]]}]}]}", DATASET_FILEPATH)

('/content/gdrive/My Drive/Dataset/dataset.zip',
 <http.client.HTTPMessage at 0x7fcce49ff898>)

One-time: Extract data from archive

In [0]:
from zipfile import ZipFile

with ZipFile(DATASET_FILEPATH, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(DIRECTORY_PATH)

# More Setup

Sets up ground truth dataset 

In [0]:
# Must be run every time!

import numpy as np

METADATA_PATH = DIRECTORY_PATH + "ISIC-images/metadata.csv"


Converts CSV file into a dictionary for classification lookups

In [13]:
import csv

with open(METADATA_PATH, mode='r') as infile:
    reader = csv.DictReader(infile)
    GROUND_TRUTH_DICT = {row['name'] : row['meta.clinical.benign_malignant'] for row in reader} 

print(GROUND_TRUTH_DICT)

{'ISIC_0000000': 'benign', 'ISIC_0000001': 'benign', 'ISIC_0000002': 'malignant', 'ISIC_0000003': 'benign', 'ISIC_0000004': 'malignant', 'ISIC_0000006': 'benign', 'ISIC_0000007': 'benign', 'ISIC_0000008': 'benign', 'ISIC_0000009': 'benign', 'ISIC_0000010': 'benign', 'ISIC_0000011': 'benign', 'ISIC_0000012': 'benign', 'ISIC_0000013': 'malignant', 'ISIC_0000014': 'benign', 'ISIC_0000015': 'benign', 'ISIC_0000016': 'benign', 'ISIC_0000017': 'benign', 'ISIC_0000018': 'benign', 'ISIC_0000019': 'benign', 'ISIC_0000020': 'benign', 'ISIC_0000021': 'benign', 'ISIC_0000022': 'malignant', 'ISIC_0000023': 'benign', 'ISIC_0000024': 'benign', 'ISIC_0000025': 'benign', 'ISIC_0000026': 'malignant', 'ISIC_0000027': 'benign', 'ISIC_0000028': 'benign', 'ISIC_0000029': 'malignant', 'ISIC_0000030': 'malignant', 'ISIC_0000031': 'malignant', 'ISIC_0000032': 'benign', 'ISIC_0000034': 'benign', 'ISIC_0000035': 'malignant', 'ISIC_0000036': 'malignant', 'ISIC_0000037': 'benign', 'ISIC_0000038': 'benign', 'ISIC_0

Sort images into propert directories

In [0]:
import shutil
import os
import pathlib

IMAGE_PATH = pathlib.Path(DIRECTORY_PATH + "ISIC-images/")
BENIGN_PATH = "/content/gdrive/My Drive/Dataset/DatasetSorted/benign/"
MALIGNANT_PATH = "/content/gdrive/My Drive/Dataset/DatasetSorted/malignant/"

pathlist = pathlib.Path(IMAGE_PATH).glob("*/*.jpg")
for path in pathlist:
    fileName = path.name
    key = GROUND_TRUTH_DICT[path.name.strip(".jpg")]
    if key == "benign":
        pastePath = BENIGN_PATH + path.name
    elif key == "malignant":
        pastePath = MALIGNANT_PATH + path.name
    else:
        print("error")
    
    copyPath = str(path)

    shutil.move(copyPath, pastePath)

**Mandatory!**

Set up some constants before we load the dataset

In [29]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)


IMAGE_PATH = pathlib.Path("/content/gdrive/My Drive/Dataset/DatasetSorted/")
IMAGE_COUNT = len(list(IMAGE_PATH.glob("*/*.jpg")))
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(IMAGE_COUNT / BATCH_SIZE)
CLASS_NAMES = np.array(["benign", "malignant"])

train_data_gen = image_generator.flow_from_directory(directory=str(IMAGE_PATH), batch_size=BATCH_SIZE, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode="categorical")
print(train_data_gen.class_indices)

Found 10532 images belonging to 3 classes.
{'.ipynb_checkpoints': 0, 'benign': 1, 'malignant': 2}
